## Wykrywanie zapalenia płuc

Próba wykorzystania zdjęć rentgenowskich klatki piersiowej do wykrywania zapalenia płuc u pacjentów (tj. przypisanie każdemu obrazowi statusu "zapalenie płuc" (pneumonia) lub "normalny" (normal))

1. Utwórz konto na Kaggle.com
2. Kliknij na swoje zdjęcie profilowe
3. Kliknij na "Konto".
4. Przewiń w dół do sekcji "API
5. Najpierw kliknij "Expire API Token". Upewnij się, że pojawi się powiadomienie o tym, że tokeny API wygasły lub nie istnieją żadne tokeny API.
6. Następnie kliknij "Create New API Token" i pobierz plik `kaggle.json`
7. Prześlij plik `kaggle.json` do google colab.


In [1]:
!pip install kaggle
!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle/kaggle.json
!chmod 600 kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip
!rm -rf chest_xray/__MACOSX
!rm -rf chest_xray/chest_xray

     |████████████████████████████████| 63 kB 819 kB/s eta 0:00:01
  Using cached python_slugify-8.0.1-py2.py3-none-any.whl (9.7 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77721 sha256=e18f672762c3b1ec7ce4afc06b0332debe6df64a0f736dac1cda3d2d0b8ea6ae
  Stored in directory: /home/przemek/.cache/pip/wheels/e6/8e/67/e07554a720a493dc6b39b30488590ba92ed45448ad0134d253
Successfully built kaggle
mkdir: cannot create directory ‘/root/.kaggle’: Permission denied
cp: failed to access '/root/.kaggle/kaggle.json': Permission denied
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  chest-xray-pneumonia.zip
  inflating: chest_xray/__MACOSX/._chest_xray  
  inflating: chest_xray/__MACOSX/chest_xray/._.DS_Store  
  inflating: chest_xray/__MACOSX/chest_xray/._test  
  inflating: chest_xray/__MACOSX/chest_xray/._train  
  inflating: chest_xra

KeyboardInterrupt: 

### Przygotowanie danych

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from imutils import paths
from sklearn.model_selection import train_test_split

def generate_dataframe(directory):
    img_paths = list(paths.list_images(directory))
    labels = ['normal' if path.find('NORMAL') > -1 else 'pn' for path in img_paths]
    return pd.DataFrame({ 'paths': img_paths, 'labels': labels })

# df contains paths to all images with corresponding labels
all_df = generate_dataframe('/home/przemek/Deep Learning/road-to-deep-learning/road-to-deep-learning/data/pneumonia-xrays/chest_xray')

train_dataset, test_dateset = train_test_split(all_df, test_size=0.2, random_state=42)

print(train_dataset['labels'].value_counts())
print(test_dateset['labels'].value_counts())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(train_dataset, test_dateset, size=224, batch=64):
    train_generator = ImageDataGenerator(
        rescale=1./255, # multiply each pixel by value
        # augmentation params
        rotation_range=5, # randomly rotate each image by 0 to 5 degrees
        width_shift_range=0.1, # shift image horizontally by 0-10%
        height_shift_range=0.1, # shift image vertically by 0-10%
        validation_split=0.2  # creates two 'subsets': training and validation
    )
    # test on the data without any transformations (original data distribution!)
    test_generator = ImageDataGenerator(rescale=1./255)

    baseargs = {
        "x_col": 'paths',
        "y_col": 'labels',
        "class_labels": ['normal', 'pn'],
        "class_mode": 'binary',  # binary classification
        "target_size": (size, size),  # we can resize the images
        "batch_size": batch, # the number of images we present to the neural network at once
        "seed": 42
    }
    train_generator_flow = train_generator.flow_from_dataframe(
        **baseargs,
        dataframe=train_dataset,  # source data frame
        subset='training')
    validation_generator_flow = train_generator.flow_from_dataframe(
        **baseargs,
        dataframe=train_dataset,
        subset='validation')
    test_generator_flow = test_generator.flow_from_dataframe(
        dataframe=test_dateset,
        shuffle=False,
        **baseargs)

    return train_generator_flow, validation_generator_flow, test_generator_flow

In [ ]:
train_generator, validation_generator, test_generator = create_generators(train_dataset, test_dateset, 224, 32)

### Model VGG16
![vgg16_architecture](https://neurohive.io/wp-content/uploads/2018/11/vgg16-neural-network.jpg)

VGG16 zawiera 13 warstw Conv2D i 3 warstwy Dense, co daje w sumie ponad 14 milionów parametrów do wytrenowania. Model ten został zaprezentowany na konkursie ImageNet w 2014 roku. Wygrał on, stąd możemy założyć, że znaleziony wtedy zestaw wag sieci potrafi dobrze klasyfikować obrazy.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.metrics import Recall, Precision, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16


In [ ]:
SIZE = 224
def make_VGGnet():
    m = VGG16(weights = 'imagenet',
              include_top = False,
              input_shape = (SIZE, SIZE, 3))
    for layer in m.layers:
        layer.trainable = False
    # perceptron part
    # vgg part basically creates compressed representation of image
    x = Flatten()(m.output)
    x = Dense(4096, activation="relu")(x)
    x = Dense(1072, activation="relu")(x)
    x = Dropout(0.2)(x)
    predictions = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=m.input, outputs=predictions)
    ## Compile and run

    adam = Adam(learning_rate=0.001)
    model.compile(optimizer=adam,
                  loss='binary_crossentropy',
                  metrics=['accuracy', Recall(name='recall'),
                           Precision(name='precision'), AUC(name='auc')])
    return model

### Callbacks

* **Tensorboard** - narzędzie wizualizacyjne do monitorowania uczenia modelu.
* **EarlyStopping** - zatrzymuje uczenie modelu przed założoną liczbą epok. Zapobiega "overfitting'owi". Jeżeli validation loss nie poprawi się przez N epok o wartość delta, wtedy uczenie modelu zakończy się wcześniej.
* **ModelCheckpoint** - zapisuje model do pliku co epokę. *save_best_only* oznacza, że zapisanie zostanie jedynie model lepszy od poprzedniego.
* **ReduceLROnPlateau** - learning rate zostanie obniżony gdy validation loss nie zmieni się przez N epok

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping
from datetime import datetime

def get_callbacks():
    logdir = (
        f'logs/scalars/vgg_{datetime.now().strftime("%m%d%Y-%H%M%S")}'
    )
    tb = TensorBoard(log_dir=logdir) # visualization tool
    es = EarlyStopping(
        monitor="val_loss",
        min_delta=1,  # model should improve by at least 1%
        patience=20,  # amount of epochs  with improvements worse than 1% until the model stops
        verbose=2,
        mode="min",
        restore_best_weights=True,  # restore the best model with the lowest validation error
    )
    mc = ModelCheckpoint(f'model_vgg.hdf5',
                         save_best_only=True,
                         monitor='val_loss',
                         mode='min')

    ## Reduce learning rate if it gets stuck in a plateau
    rlr = ReduceLROnPlateau(monitor='val_loss',
                            factor=0.3,
                            patience=3,
                            min_lr=0.000001,
                            verbose=1)
    return [tb, es, mc, rlr]

### Trenowanie modelu

Model posiada ~14 mln parametrów pobranych z 'imagenet VGG' oraz ~107 mln części klasyfikacyjnej.

In [ ]:
from tensorflow.keras.utils import plot_model

def fit_model(train_generator, validation_generator, batch_size=32, epochs=15):
    model = make_VGGnet()
    model.summary()
    plot_model(model, to_file='vgg16.jpg', show_shapes=True)

    model_history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_steps=validation_generator.n/batch_size,
                              epochs=epochs,
                              verbose=1,
                              callbacks=get_callbacks())  # set up callbacks
    return model, model_history

In [ ]:
vgg_model, vgg_model_hist = fit_model(train_generator, validation_generator, epochs=15)

In [ ]:
def plot_history(history):

    fig = plt.figure(figsize = (18 , 6))

    fig.add_subplot(1,2,1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train loss', 'valid loss'])
    plt.grid(True)
    plt.plot()

    fig.add_subplot(1,2,2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['train acc', 'valid acc'])
    plt.grid(True)
    plt.plot()

In [ ]:
plot_history(vgg_model_hist)

### Tensorboard

Narzędzie pozwalające na monitorowanie procesu trenowania lub jego wyników.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

### Wizualizacja atencji modelu z GradCAM

Grad-CAM jest algorytmem, który pozwala nam generować mapy termiczne, które pokazują które części obrazu miały największy wpływ na ostateczną decyzję sieci.

In [ ]:
!pip install scikit-image

In [ ]:
!git clone https://github.com/gkeechin/vizgradcam.git
!cp vizgradcam/gradcam.py gradcam.py

In [ ]:
from tensorflow.keras.models import load_model
from gradcam import VizGradCAM

def display_map_and_conf(model, test_generator):
    imgs = test_generator.next()
    fig = plt.figure(figsize=(15,5), facecolor='white')

    for i in range(3):
        fig.add_subplot(1,3,i+1)
        image = imgs[0][i]
        label = 'PNEUMONIA' if imgs[1][i] == 1 else 'NORMAL'
        VizGradCAM(model, image, plot_results=True, interpolant=0.5)
        out_prob = model.predict(image.reshape(1,224,224,3))[0][0]
        title = f"Prediction: {'PNEUMONIA' if out_prob > 0.5 else 'NORMAL'}\n"
        title += f"Prob(Pneumonia): {out_prob}\n"
        title += f"True Label: {label}\n"
        plt.title(title)

In [ ]:
loaded_model = load_model('model_vgg.hdf5')
display_map_and_conf(loaded_model, test_generator)

### Predykcja

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import numpy as np

def predict_image(model, img_path):
    img = img_to_array(load_img(img_path, target_size=(224,224,3)))
    img = img * (1./255)
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    label = 'PNEUMONIA' if pred >= 0.5 else 'NORMAL'
    print("prediction: ", label, "P(Pneumonia): ", pred[0][0])

In [ ]:
m = load_model('model_vgg.hdf5')
predict_image(m, '/home/przemek/Deep Learning/road-to-deep-learning/road-to-deep-learning/data/pneumonia-xrays/chest_xray/train/NORMAL/IM-0410-0001.jpeg')

### Zadanie

* Opisz na czym polega **transfer learning**. Jak został on wykorzystany w powyższym przykładzie?
* Dokonaj ewaluacji modelu VGG16. Użyj [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) oraz [ConfusionMatrixDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html). Wyznacz wartości **Precision** oraz **Recall**.